In [11]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split

from fastai.collab import *
from fastai.tabular import *
from fastai.tabular.all import *
from fastai.callback import *

import warnings
warnings.filterwarnings("ignore")

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## Load Datasets

In [384]:
# info on business_id with business_id_cat and suplement infos
philly_complete_df = pd.read_csv('business_id_with_num_id_complete.csv').drop('Unnamed: 0',axis=1)

In [415]:
philly_reviews_df = pd.read_csv('vader_sent_filtered_philly_295k.csv')

# actual reviews on philadelphia
philly_reviews_df_cut = philly_reviews_df[['user_id_cat','business_id_cat','stars']]

In [386]:
# test set
test_df = pd.read_csv('user_test_df.csv')

# Load Models

In [387]:
learn = load_learner('tab_best_model.pkl')

In [388]:
def calculate_test_metrics(learn_model, model_name):
    test_dl = learn_model.dls.test_dl(test_df)
    sub = learn_model.get_preds(dl=test_dl)

    # Compute the test RMSE
    y_preds = sub[0].numpy()
    y_true = sub[1].numpy()
    test_rmse = mean_squared_error(y_true, y_preds) ** 0.5
    print(f'{model_name} Test RMSE: {test_rmse}' )
    test_mae = mean_absolute_error(y_true, y_preds)
    print(f'{model_name} Test MAE: {test_mae}')
    
    return (test_rmse, test_mae)

best_test_metrics = calculate_test_metrics(learn, 'Best Params Tabular Learner Model')

Best Params Tabular Learner Model Test RMSE: 0.9590482671084907
Best Params Tabular Learner Model Test MAE: 0.7451607584953308


In [389]:
user_resto_count = test_df.groupby('user_id_cat')['business_id_cat'].count()
user_resto_count = user_resto_count.sort_values(ascending=False)[:10]
user_resto_count

user_id_cat
220     134
606     123
438     101
170      92
434      87
517      82
450      77
1410     76
590      73
389      72
Name: business_id_cat, dtype: int64

In [401]:
def get_top_k_recommendations_tabular(model, user_id_cat, k):
    
    user_test_df = test_df[test_df['user_id_cat'] == user_id_cat]
    user_test_dl = model.dls.test_dl(user_test_df)
    sub = model.get_preds(dl=user_test_dl)
    user_star_preds = sub[0].numpy()
    user_recom_df = user_test_df.copy()[['user_id_cat', 'business_id_cat']]
    user_recom_df['stars_preds'] = user_star_preds
    top_k_recom_df = user_recom_df.sort_values('stars_preds', ascending=False)[:k]
    
    return top_k_recom_df

In [402]:
import random

In [473]:
top_10_users_who_rate = user_resto_count.index
# top_10_users_who_rate = [220]

In [423]:
# k = 10 # get max users
# recommendations_tabular = get_top_k_recommendations_tabular(learn, test_user_id, k).sort_values(by='stars_preds',ascending=False) # get predictions

In [424]:
def get_info_restaurants(philly_complete_df, restaurant_ids):
    return philly_complete_df[philly_complete_df['business_id_cat'].isin(restaurant_ids)][['business_id','business_id_cat','name','categories']]

In [445]:
def get_final_comparison(test_user_id, model, k, philly_reviews_df_cut, philly_complete_df):
    # recommendations
    recommendations_tabular = get_top_k_recommendations_tabular(learn, test_user_id, k).sort_values(by='stars_preds',ascending=False) # get predictions 
    
    # restaurant_id_cat of best k restaurant by user
    users_top_res_pred_ids = (recommendations_tabular['business_id_cat'].values)     
    
    # actual reviews by user
    philly_reviews_df_user = philly_reviews_df_cut[philly_reviews_df_cut['user_id_cat'] == test_user_id].sort_values(by='stars',ascending=False)
    actual_stars = philly_reviews_df_user[philly_reviews_df_user['business_id_cat'].isin(users_top_res_pred_ids)].drop('user_id_cat',axis=1)
    
    # info on best k restaurant predicted by user
    users_resto_pred = get_info_restaurants(philly_complete_df, users_top_res_pred_ids)
    
    # prediction + info on restaurants
    users_pred_full = pd.merge(users_resto_pred, recommendations_tabular, on='business_id_cat', how='inner')[['user_id_cat','business_id_cat','name','categories','stars_preds']]
    
    return pd.merge(users_pred_full,actual_stars, on='business_id_cat', how='inner').sort_values(by='stars_preds', ascending=False)

In [454]:
test_user_id = random.choice(top_10_users_who_rate) #choose random user
get_final_comparison(test_user_id, learn, 10, philly_reviews_df_cut, philly_complete_df)

,user_id_cat,business_id_cat,name,categories,stars_preds,stars
2,134,619,John's Roast Pork,"Sandwiches, Salad, American (Traditional), Cheesesteaks, Breakfast & Brunch, Active Life, American (New), Beaches, Restaurants",4.323034,4.0
6,134,387,Bobo's Sports Bar,"Restaurants, Nightlife, Karaoke, Korean, Bars",4.190826,4.0
0,134,651,Fogo de Chao,"Seafood, Steakhouses, Brazilian, Restaurants",4.022256,3.0
3,134,4060,Sabrina's Café,"Vegetarian, Breakfast & Brunch, American (New), Coffee & Tea, American (Traditional), Caterers, Food, Cafes, Event Planning & Services, Restaurants, Diners",3.986262,4.0
5,134,202,Capogiro Gelato,"Pizza, Restaurants, Gelato, Coffee & Tea, Ice Cream & Frozen Yogurt, Food",3.955893,4.0
9,134,2594,Yong Hwa Roo,"Korean, Restaurants",3.893948,3.0
1,134,1784,Pho Saigon,"Coffee & Tea, Beauty & Spas, Cosmetics & Beauty Supply, Shopping, Arts & Crafts, Costumes, Restaurants, Vietnamese, Food",3.892079,4.0
4,134,5256,Sketch Burger,"Restaurants, Burgers",3.848879,4.0
8,134,2467,Tai's Vietnamese Food,"Food Stands, Restaurants, Vietnamese",3.823022,4.0
7,134,2003,Pub Webb,"Beer, Wine & Spirits, Food, Nightlife, Restaurants, Music Venues, Sports Bars, Bars, Arts & Entertainment, American (New)",3.751527,3.0


In [474]:
top_10_users_who_rate

Int64Index([220, 606, 438, 170, 434, 517, 450, 1410, 590, 389], dtype='int64', name='user_id_cat')

In [475]:
get_final_comparison(top_10_users_who_rate[0], learn, 10, philly_reviews_df_cut, philly_complete_df)

,user_id_cat,business_id_cat,name,categories,stars_preds,stars
7,220,4865,Palm Tree Gourmet,"Restaurants, Vegetarian, Grocery, Specialty Food, Sandwiches, Food",4.525239,5.0
3,220,61,Tria Cafe Wash West,"Beer, Wine & Spirits, Wine Bars, Modern European, American (New), Cheese Shops, Specialty Food, Food, Restaurants, Bars, Nightlife",4.511053,5.0
9,220,2361,Square Pie,"Italian, Pizza, Restaurants",4.488066,5.0
2,220,2506,Bardot Cafe,"French, Cocktail Bars, Wine Bars, Lounges, American (New), Cafes, Beer Bar, Bars, Nightlife, Restaurants",4.473537,4.0
0,220,2405,Frangelli's Bakery,"Restaurants, Food, Donuts, Bakeries",4.459857,5.0
4,220,685,Chapterhouse Café & Gallery,"Restaurants, Food, Coffee & Tea",4.448271,5.0
6,220,1911,Porto,"Breakfast & Brunch, Portuguese, Restaurants, Food, Coffee & Tea",4.442429,4.0
8,220,781,Pita Chip,"Mediterranean, Vegetarian, Sandwiches, Wraps, Halal, Restaurants",4.434885,5.0
1,220,1819,PHS Pop Up Garden,"Parks, Active Life, Beer Gardens, Mexican, Local Flavor, Restaurants, Nightlife, Bars, Beer, Wine & Spirits, Food",4.429041,5.0
5,220,3469,Root,"Wine Bars, Nightlife, Bars, American (New), Restaurants, Italian, Spanish",4.424079,5.0


In [476]:
get_final_comparison(top_10_users_who_rate[1], learn, 10, philly_reviews_df_cut, philly_complete_df)

,user_id_cat,business_id_cat,name,categories,stars_preds,stars
7,606,5374,Paesano's Philly Style,"Restaurants, Sandwiches, Italian",4.609363,5.0
8,606,4847,Artisan Boulanger Patissier,"Food, Restaurants, Bakeries",4.573194,5.0
4,606,3690,Rival Bros Coffee,"Restaurants, Coffee Roasteries, Food, Cafes, Coffee & Tea",4.560959,5.0
9,606,5007,Teaful Bliss Cafe,"Restaurants, Coffee & Tea, Food, Sandwiches, Tea Rooms",4.541941,5.0
3,606,1249,Mama's Vegetarian,"Comfort Food, Middle Eastern, Vegetarian, Kosher, Restaurants",4.541305,5.0
1,606,3596,Mike's BBQ,"Event Planning & Services, Barbeque, Sandwiches, Restaurants, Caterers",4.532125,5.0
2,606,796,Ray's Café & Teahouse,"Coffee & Tea, Chinese, Restaurants, Food",4.526658,5.0
6,606,2977,Salumeria,"Food, Delis, Sandwiches, Cheese Shops, Restaurants, Specialty Food",4.521232,3.0
0,606,4775,Grindcore House,"Coffee & Tea, Vegetarian, Food, Restaurants, Vegan",4.507734,5.0
5,606,3994,Forsythia,"Restaurants, French, Modern European, American (New)",4.500880,5.0


In [477]:
get_final_comparison(top_10_users_who_rate[2], learn, 10, philly_reviews_df_cut, philly_complete_df)

,user_id_cat,business_id_cat,name,categories,stars_preds,stars
3,438,190,Suraya,"Active Life, Beer Gardens, Grocery, Middle Eastern, Specialty Food, American (New), Parks, Cafes, Food, Nightlife, Restaurants, Coffee & Tea, Lebanese",4.683440,4.0
7,438,3098,Jansen,"American (New), Restaurants",4.595718,5.0
5,438,4670,Los Gallos Mexican Taqueria,"Mexican, Restaurants, Food",4.586798,5.0
2,438,5261,Wm Mulherin's Sons,"Restaurants, Tapas/Small Plates, Italian, Szechuan, Cocktail Bars, Pizza, Chinese, Bars, Nightlife",4.581764,4.0
1,438,794,Townsend,"French, Nightlife, Restaurants, Bars, Cocktail Bars",4.559340,3.0
4,438,988,T & F Farmers Pride,"Specialty Food, Delis, Sandwiches, Fruits & Veggies, International Grocery, Grocery, Food, Restaurants, Farmers Market",4.556582,5.0
0,438,737,V Street,"American (New), Breakfast & Brunch, Vegetarian, Vegan, Bars, Wine Bars, Restaurants, Nightlife",4.556120,3.0
9,438,4693,Barry's Steaks,"Chicken Wings, Restaurants, Burgers, Sandwiches, Cheesesteaks, Steakhouses",4.547423,3.0
6,438,5329,Stockyard Sandwich,"Restaurants, Sandwiches, Caterers, Gluten-Free, Event Planning & Services, Breakfast & Brunch, Salad",4.544025,5.0
8,438,2593,Rione,"Food, Coffee & Tea, Pizza, Restaurants, Italian",4.543732,5.0


In [478]:
get_final_comparison(top_10_users_who_rate[3], learn, 10, philly_reviews_df_cut, philly_complete_df)

,user_id_cat,business_id_cat,name,categories,stars_preds,stars
0,170,1201,Zahav,"Nightlife, Bars, Food, Ethnic Food, Middle Eastern, Vegetarian, Specialty Food, Israeli, Restaurants",4.217649,5.0
1,170,2448,Philadelphia Museum of Art,"Art Museums, Jazz & Blues, Arts & Crafts, American (New), Shopping, Museums, Nightlife, Art Galleries, Restaurants, Arts & Entertainment",4.183995,5.0
8,170,5242,Vedge,"Restaurants, Vegetarian, Beer, Wine & Spirits, Vegan, Food",4.182195,4.0
5,170,3517,Talula's Garden,"American (New), Restaurants",4.176775,4.0
7,170,50,Octopus Falafel Truck,"Food Trucks, Falafel, Food Stands, Food, Mediterranean, Restaurants, Middle Eastern",4.126738,4.0
3,170,1771,Nook Bakery & Coffee Bar,"Event Planning & Services, Venues & Event Spaces, Cupcakes, Coffee Roasteries, Italian, Food, Bakeries, Coffee & Tea, Restaurants",4.030988,4.0
2,170,5250,Philadelphia's 9th Street Italian Market,"Specialty Food, Local Flavor, Public Markets, Fruits & Veggies, Grocery, Food, Shopping, Farmers Market, Restaurants",3.951572,5.0
9,170,3066,Independence Visitor Center,"Breakfast & Brunch, Landmarks & Historical Buildings, Hotels & Travel, Local Flavor, Museums, Public Services & Government, Restaurants, Travel Services, Arts & Entertainment, Ticket Sales, Visitor Centers",3.948242,5.0
6,170,4116,Double Knot,"Nightlife, Sushi Bars, Cafes, Japanese, Bars, Restaurants",3.930596,4.0
4,170,3688,Stock's Bakery,"Restaurants, Food, Bakeries",3.916150,4.0


In [479]:
get_final_comparison(top_10_users_who_rate[4], learn, 10, philly_reviews_df_cut, philly_complete_df)

,user_id_cat,business_id_cat,name,categories,stars_preds,stars
9,434,4303,Hikari Sushi,"Restaurants, Sushi Bars, Japanese",4.108535,5.0
3,434,3558,Vernick Food & Drink,"Food, Restaurants, Beer, Wine & Spirits, Diners, American (New)",4.090451,4.0
5,434,5273,Cafe La Maude,"Sandwiches, Cafes, Coffee & Tea, Restaurants, Food, Breakfast & Brunch, Mediterranean, American (New)",4.085533,5.0
7,434,2503,Love & Honey Fried Chicken,"Restaurants, Chicken Shop, Chicken Wings, Sandwiches, Comfort Food, American (Traditional), Soul Food",4.052926,5.0
0,434,2364,Barbuzzo,"Mediterranean, Restaurants, Pizza, Italian",4.026815,4.0
8,434,556,Bistro La Bête,"French, Restaurants",4.001869,5.0
2,434,619,John's Roast Pork,"Sandwiches, Salad, American (Traditional), Cheesesteaks, Breakfast & Brunch, Active Life, American (New), Beaches, Restaurants",3.998363,4.0
6,434,1962,Ken Love's BYOB,"Vegetarian, Mediterranean, Desserts, Food, American (New), Restaurants, Steakhouses",3.989999,4.0
1,434,3548,Gaul & Co. Malthouse,"Sandwiches, Restaurants, Nightlife, American (New), Bars, Beer Bar",3.980567,4.0
4,434,5261,Wm Mulherin's Sons,"Restaurants, Tapas/Small Plates, Italian, Szechuan, Cocktail Bars, Pizza, Chinese, Bars, Nightlife",3.958435,5.0


In [480]:
get_final_comparison(top_10_users_who_rate[5], learn, 10, philly_reviews_df_cut, philly_complete_df)

,user_id_cat,business_id_cat,name,categories,stars_preds,stars
8,517,3421,A La Mousse,"Coffee & Tea, Desserts, Food, Restaurants, Cafes",4.598543,5.0
0,517,619,John's Roast Pork,"Sandwiches, Salad, American (Traditional), Cheesesteaks, Breakfast & Brunch, Active Life, American (New), Beaches, Restaurants",4.544319,5.0
9,517,2837,Sutton's,"American (Traditional), Cocktail Bars, Restaurants, Nightlife, Bars",4.505793,4.0
2,517,717,IndeBlue Modern Indian Food & Spirits,"Cocktail Bars, Food Delivery Services, Nightlife, Breakfast & Brunch, Food, Bars, Event Planning & Services, Caterers, Restaurants, Indian",4.484790,5.0
7,517,4090,Miles Table,"Restaurants, Coffee & Tea, Food, Breakfast & Brunch, Event Planning & Services, American (Traditional), Caterers",4.480770,5.0
5,517,4723,JJ Thai Cuisine,"French, Restaurants, Thai, Asian Fusion",4.478360,4.0
6,517,1221,&pizza - Walnut,"Pizza, Vegetarian, Restaurants, Vegan",4.444690,5.0
1,517,2950,Dutch Eating Place,"Sandwiches, Restaurants, Juice Bars & Smoothies, Food, Breakfast & Brunch",4.436519,4.0
3,517,1807,Kawaii Kitty Cafe,"Pet Adoption, Vegetarian, Restaurants, Cafes, Japanese, Pets, Coffee & Tea, Food, Vegan",4.427084,5.0
4,517,5136,Chubby Cattle,"Restaurants, Bars, Seafood, Japanese, Hot Pot, Nightlife, Chinese, Conveyor Belt Sushi, Mongolian, Asian Fusion",4.421712,5.0


In [481]:
get_final_comparison(top_10_users_who_rate[6], learn, 10, philly_reviews_df_cut, philly_complete_df)

,user_id_cat,business_id_cat,name,categories,stars_preds,stars
5,450,5242,Vedge,"Restaurants, Vegetarian, Beer, Wine & Spirits, Vegan, Food",4.422614,5.0
1,450,5224,Reading Terminal Market,"Candy Stores, Shopping, Department Stores, Fast Food, Beer, Wine & Spirits, Fruits & Veggies, Chinese, Food, Ice Cream & Frozen Yogurt, Desserts, Seafood, Health Markets, Bagels, Cheese Shops, Shopping Centers, Chocolatiers & Shops, Meat Shops, Public Markets, Food Court, Wineries, Local Flavor, Ethnic Food, Restaurants, Specialty Food, Arts & Entertainment, Juice Bars & Smoothies, Seafood Markets, Farmers Market, Coffee & Tea, Bakeries, Food Stands, Dinner Theater, Sporting Goods, Grocery, Fashion",4.407960,5.0
9,450,830,Marchiano's Bakery,"Bakeries, Restaurants, Food",4.378778,5.0
0,450,2364,Barbuzzo,"Mediterranean, Restaurants, Pizza, Italian",4.367835,4.0
2,450,625,Butcher and Singer,"Restaurants, Steakhouses, American (Traditional)",4.347415,5.0
8,450,4766,McNally's Tavern,"Nightlife, Sandwiches, Restaurants, Bars, American (Traditional), Pubs, Food, Cheesesteaks, Beer, Wine & Spirits",4.310632,4.0
3,450,4746,Fond,"American (New), Bars, Seafood, Restaurants, Nightlife",4.306818,5.0
6,450,4723,JJ Thai Cuisine,"French, Restaurants, Thai, Asian Fusion",4.296967,5.0
7,450,4170,Magpie,"Desserts, Food, Coffee & Tea, Restaurants",4.287242,4.0
4,450,1780,Elixr Coffee Roasters,"Cafes, Food, Coffee & Tea, Bakeries, Coffee Roasteries, Restaurants",4.285556,5.0


In [482]:
get_final_comparison(top_10_users_who_rate[7], learn, 10, philly_reviews_df_cut, philly_complete_df)

,user_id_cat,business_id_cat,name,categories,stars_preds,stars
1,1410,4649,Beiler's Bakery,"Food, Donuts, Food Stands, Restaurants, Specialty Food, Bakeries, Breakfast & Brunch",4.655671,5.0
4,1410,3558,Vernick Food & Drink,"Food, Restaurants, Beer, Wine & Spirits, Diners, American (New)",4.582487,4.0
0,1410,3596,Mike's BBQ,"Event Planning & Services, Barbeque, Sandwiches, Restaurants, Caterers",4.524184,5.0
7,1410,3742,Sarcone’s Bakery,"Bakeries, Food, Restaurants",4.497868,4.0
3,1410,3534,Termini Bros Bakery,"Bakeries, Restaurants, Food, Desserts",4.493956,5.0
2,1410,2436,Knead Bagels,"Restaurants, Bagels, Coffee & Tea, Sandwiches, Food, Soup, Salad",4.480201,3.0
5,1410,1235,Gran Caffe L'Aquila,"Restaurants, Gelato, Coffee & Tea, Food, Italian, Bakeries",4.465431,5.0
9,1410,935,Rieker's Prime Meats,"Meat Shops, Food, Restaurants, Butcher, Grocery, Specialty Food, German",4.455945,5.0
8,1410,1022,Carangi Baking,"Bakeries, Cafes, Food, Sandwiches, Restaurants, Italian",4.450222,5.0
6,1410,2843,Falafel Time,"Restaurants, Mediterranean, Middle Eastern, Halal",4.449414,4.0


In [483]:
get_final_comparison(top_10_users_who_rate[8], learn, 10, philly_reviews_df_cut, philly_complete_df)

,user_id_cat,business_id_cat,name,categories,stars_preds,stars
9,590,4303,Hikari Sushi,"Restaurants, Sushi Bars, Japanese",4.351129,4.0
2,590,4126,Saad's Halal Restaurant,"Halal, Middle Eastern, Restaurants, Food, Indian, Cheesesteaks, Desserts",4.347386,4.0
7,590,768,Rybrew,"Restaurants, Beer, Wine & Spirits, Salad, Sandwiches, Food, American (New)",4.236288,3.0
1,590,2405,Frangelli's Bakery,"Restaurants, Food, Donuts, Bakeries",4.231188,5.0
8,590,1022,Carangi Baking,"Bakeries, Cafes, Food, Sandwiches, Restaurants, Italian",4.164796,4.0
6,590,4695,Greenstreet Coffee Co.,"Restaurants, Food, Cafes, Coffee & Tea",4.153143,4.0
4,590,5375,Pizza Shackamaxon,"Restaurants, Fast Food, Pizza",4.109270,4.0
3,590,679,SouthHouse,"Bars, American (New), Nightlife, Restaurants, Pubs, Gluten-Free",4.082190,4.0
0,590,280,The Freshworks of Port Richmond,"Delis, Salad, Cheesesteaks, Wraps, Restaurants, Sandwiches",4.057274,5.0
5,590,694,Tacconelli's Pizzeria,"Pizza, Restaurants",4.043817,3.0


In [512]:
get_final_comparison(top_10_users_who_rate[9], learn, 10, philly_reviews_df_cut, philly_complete_df).drop('user_id_cat',axis=1).reset_index(drop=True)

,business_id_cat,name,categories,stars_preds,stars
0,1201,Zahav,"Nightlife, Bars, Food, Ethnic Food, Middle Eastern, Vegetarian, Specialty Food, Israeli, Restaurants",4.265881,3.0
1,757,Sky Cafe,"Noodles, Restaurants, Indonesian",4.163609,5.0
2,648,Daleng Restaurant,"Restaurants, Cambodian, Thai, Vietnamese, Asian Fusion",4.058896,4.0
3,4746,Fond,"American (New), Bars, Seafood, Restaurants, Nightlife",4.043224,4.0
4,5250,Philadelphia's 9th Street Italian Market,"Specialty Food, Local Flavor, Public Markets, Fruits & Veggies, Grocery, Food, Shopping, Farmers Market, Restaurants",3.999816,5.0
5,2962,Rex at the Royal,"Southern, Restaurants, Seafood, Salad, American (New)",3.983941,4.0
6,3570,The Grill Smuckers,"Restaurants, American (New), Breakfast & Brunch, Farmers Market, Food",3.965728,4.0
7,33,Zorba's Taverna,"Restaurants, Greek, Mediterranean",3.948534,4.0
8,1718,Via Locusta,"Bars, Tapas/Small Plates, Cocktail Bars, Nightlife, Italian, Restaurants",3.939758,4.0
9,3127,Isot Mediterranean Cuisine,"Restaurants, Middle Eastern, Breakfast & Brunch, Turkish, Mediterranean, Halal",3.903615,4.0


In [485]:
train_df = pd.read_csv('user_train_df.csv')

In [488]:
train_df_cut = train_df[['user_id_cat','business_id_cat','stars']]

In [490]:
train_df_cut_user_389 = train_df_cut[train_df_cut['user_id_cat'] == 389]

In [494]:
restaurants_user_389_train = train_df_cut_user_389['business_id_cat'].values

In [498]:
restaurant_info_389 = get_info_restaurants(philly_complete_df, restaurants_user_389_train)

In [504]:
train_389_rates = pd.merge(train_df_cut_user_389, restaurant_info_389, on='business_id_cat',how='inner').drop('business_id',axis=1).sort_values(by='stars',ascending=False)

In [505]:
train_389_rates.columns

Index(['user_id_cat', 'business_id_cat', 'stars', 'name', 'categories'], dtype='object')

In [506]:
train_389_rates = train_389_rates[['user_id_cat', 'business_id_cat', 'name', 'categories','stars']]

In [509]:
train_389_rates.drop('user_id_cat',axis=1)

,business_id_cat,name,categories,stars
18,3495,The Dandelion,"British, Restaurants, Gastropubs, Bars, Nightlife",4.0
11,2522,Chili Szechuan,"Restaurants, Szechuan, Chinese",4.0
22,1220,Zavino,"Italian, Wine Bars, Bars, Mediterranean, Pizza, Restaurants, Nightlife",4.0
21,5221,Green Eggs Café,"Restaurants, American (New), Diners, Breakfast & Brunch",4.0
27,643,Bistrot La Minette,"Restaurants, French, Breakfast & Brunch",4.0
19,190,Suraya,"Active Life, Beer Gardens, Grocery, Middle Eastern, Specialty Food, American (New), Parks, Cafes, Food, Nightlife, Restaurants, Coffee & Tea, Lebanese",4.0
1,4102,Bufad,"Desserts, Event Planning & Services, Caterers, Vegetarian, Pizza, Restaurants, Food, Italian, Venues & Event Spaces",4.0
15,2405,Frangelli's Bakery,"Restaurants, Food, Donuts, Bakeries",4.0
14,211,Slices Pizza,"Pizza, Food, Restaurants",4.0
13,2410,Pizza Brain,"Restaurants, Pizza, Specialty Food, Museums, Food, Arts & Entertainment",4.0
